# AG002 - Lucas Fajardo de Mello e Arthur de Paula Assis

Para começarmos a AG primeiramente temos que carregar o nosso dataset, para isso vamos utilizar a biblioteca pandas, com a função read_csv.

Vamos também importar algumas bibliotecas que utilizaremos depois.


In [1]:
import pandas as pd

dataset = pd.read_csv("Dataset/Wholesale customers data.csv")
dataset.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


## Ex 3

A versão baixada do link disponibilizado já estava com uma mudança similar a mudança proposta realizada, porém ná próxima célula de código iremos demonstrar como a mudança seria realizada caso necessário

In [2]:
dataset.Channel.replace({"HoReCa":0,"Retail":1}, inplace = True) #Altera os valores HoReCa e Retail para 0 e 1
dataset.Channel.replace({1:0,2:1}, inplace = True) #Altera os valores que estavam presentes no dataset pelos pedidos no exercicio

dataset.Region.replace({"Lisbon":0,"Oporto":1,"Other":2}, inplace = True) #Altera os valores Lisbon, Oporto e Other para 0, 1 e 2
dataset.Region.replace({1:0,2:1,3:2}, inplace = True) #Altera os valores que estavam presentes no dataset pelos pedidos no exercicio
dataset.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,1,2,12669,9656,7561,214,2674,1338
1,1,2,7057,9810,9568,1762,3293,1776
2,1,2,6353,8808,7684,2405,3516,7844
3,0,2,13265,1196,4221,6404,507,1788
4,1,2,22615,5410,7198,3915,1777,5185


## Ex 4

Nesse exercício iremos mudar a ordem das colunas do nosso dataframe, para isso iremos utilizar um artifício onde ao chamarmos um dataframe com uma lista das colunas desejadas em uma certa ordem o pandas retorna um dataframe com as colunas nessa mesma ordem, assim podemos atribuir esse dataframe reordenado ao original, assim o reorganizando sem a necessidade de chamadas de função mais complexas

In [3]:
novaOrdem =['Region', 'Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicassen', 'Channel'] #Estabelecemos um array um com os valores na ordem nova dessejada

dataset = dataset [novaOrdem] #Altera a ordem para a nova ordem desejada

dataset.head()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel
0,2,12669,9656,7561,214,2674,1338,1
1,2,7057,9810,9568,1762,3293,1776,1
2,2,6353,8808,7684,2405,3516,7844,1
3,2,13265,1196,4221,6404,507,1788,0
4,2,22615,5410,7198,3915,1777,5185,1


## Ex 5

Nesse exercício iremos separar nosso dataset em uma seção de teste e outra para treinos do nosso modelo de ML, iremos utilizar um split de 80/20.

Para realizar essa divisão utilizaremos a função train_test_split do sklearn

In [4]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2)

## Ex 6 

Nesse exercício iremos escolher um modelo de ML, decidimos por escolher o modelo Naive Bayes, por ser um modelo simples baseado na matemática e que é amplamente utilizado para predições similares no contexto de varejo.

Antes de treinar o nosso modelo vamos também utilizar um algoritmo de Forward Selection com K-fold cross validation para escolher apenas as features que terão um impacto significativo no nosso modelo

In [43]:
from sklearn.model_selection import cross_val_score

def K_fold_validation_scoring(model,X,y,scoring:str):
    scores = cross_val_score(model, X, y, cv=10,scoring=scoring)#does 10-fold cross validation and returns the NRMSE scores
    return scores.mean()#returns the mean NRMSE value

In [44]:
def fwd_selection_cross_validation(X,y,features,minScoreVariation : float,detailedOutput : bool,scoring : str,model):
    used_features=[]#list to store the features should be used on our model according to forward selection 
    unused_features = list(features)#list of all the features available to our model, in the end will contain the unused features
    current_score = float('-inf')#variable to store the score of our curent model
    best_current_score = float('-inf')#variable to store the best score recorded

    while current_score == best_current_score and unused_features:
        score_for_feature = []#list to store the score obtained with each feature
        for feature in unused_features:#iterates trough every unused feature and does 10-fold cross validation for each
            current_features = used_features + [feature]
            model.fit(X=X[current_features],y=y)
            score = K_fold_validation_scoring(model,X[current_features],y,scoring)
            score_for_feature.append((score,feature))
        
        score_for_feature.sort()#sorts the scores so the lowest value is on the beginning of the array
        best_current_score,best_feature = score_for_feature.pop(score_for_feature.__len__()-1)#gets the best feature along with its score

        if(detailedOutput):#if detailed output is set to True outputs more detailed information about the process
            print('Best current score: '+str(best_current_score)+'\nWith addition of feature: '+str(best_feature))

        if current_score < (best_current_score-minScoreVariation):#if the new score - the minimun varition set by the user
            unused_features.remove(best_feature)# is lower than the precious score, adds the feature to the selected features
            used_features.append(best_feature)# and replaces the current score with the new score
            current_score = best_current_score
        elif(detailedOutput):
                print('Rejected feature: '+best_feature+'\nReason : Low score increase (<'+str(minScoreVariation)+')\nFinal model has '+str(used_features.__len__())+' features')

    return used_features,current_score
    
#O código de ambas as funções foram feitas por mim Lucas Fajardo, os comentários apenas estao em inglês pois fiz eles para um projeto durante meu intercâmbio

In [61]:
from time import time
from sklearn.naive_bayes import GaussianNB

features = train_data.columns.drop("Channel") #Seleciona apenas a feature a serem utilizadas pelo modelo

start = time() # Tempo inicial
selected_features,acc = fwd_selection_cross_validation(X=train_data[features],y=train_data['Channel'],features=features,minScoreVariation=0.01,detailedOutput=True,scoring = 'accuracy',model= GaussianNB())
elapsedTime = time()-start # Tempo total

print('Selected features: '+str(selected_features)+'\nAccuracy: '+str(acc)+'\nIn '+str(elapsedTime)+' Seconds')

Best current score: 0.9034126984126984
With addition of feature: Detergents_Paper
Best current score: 0.9062698412698413
With addition of feature: Region
Rejected feature: Region
Reason : Low score increase (<0.01)
Final model has 1 features
Selected features: ['Detergents_Paper']
Accuracy: 0.9034126984126984
In 0.28357553482055664 Seconds


O nosso modelo está apresentando uma precisão de 90.34% com apenas a feature Detergents_Paper, isso é altamente incomum, e mais pra frente com mais métricas iremos analizar isso mais a fundo

## EX 7

Agora iremos testar a precisão do nosso modelo com os nossos dados de teste.

In [51]:
NBmodel = GaussianNB()

NBmodel.fit(train_data[selected_features],y=train_data['Channel'])

Predictions = NBmodel.predict(test_data[selected_features])
Predictions[:10]

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0], dtype=int64)

Vamos também treinar um modelo com todas as features disponíveis para ver a diferença nas métricas

In [52]:
NBmodelFull = GaussianNB()

NBmodelFull.fit(train_data[features],y=train_data['Channel'])

PredictionsFull = NBmodelFull.predict(test_data[features])
PredictionsFull[:10]

array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0], dtype=int64)

## Ex 8

Nesse exercício iremos avaliar os nossos modelos com base em diversas métricas, a maioria obtida através da função classification_report além da acurácia dos modelos e a confusion matrix

In [58]:
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix

print(classification_report(test_data["Channel"],Predictions,target_names=["HoTeCa","Retail"]))
testAcc = accuracy_score(test_data['Channel'], Predictions)
print('Accuracy: '+str(testAcc))
confusionMat = confusion_matrix(test_data['Channel'], Predictions)
print('Confusion matrix:\n')
print(confusionMat)

              precision    recall  f1-score   support

      HoTeCa       0.94      0.94      0.94        65
      Retail       0.83      0.83      0.83        23

    accuracy                           0.91        88
   macro avg       0.88      0.88      0.88        88
weighted avg       0.91      0.91      0.91        88

Accuracy: 0.9090909090909091
Confusion matrix:

[[61  4]
 [ 4 19]]


No geral obtivemos boas métricas, com uma precisão acima de 90%, sendo excelende para um modelo tão simples quanto o naive bayes, ainda mais com apenas uma feature, além de recall e F1 próximos de 1 nos indicando que o nosso modelo não conseguiu métricas boas apenas "chutando" o mesmo valor toda vez, informação que conseguimos comprovar através da confusion matrix

In [59]:
print(classification_report(test_data["Channel"],PredictionsFull,target_names=["HoTeCa","Retail"]))
testAcc = accuracy_score(test_data['Channel'], PredictionsFull)
print('Accuracy: '+str(testAcc))
confusionMat = confusion_matrix(test_data['Channel'], PredictionsFull)
print('Confusion matrix:\n')
print(confusionMat)

              precision    recall  f1-score   support

      HoTeCa       0.94      0.94      0.94        65
      Retail       0.83      0.83      0.83        23

    accuracy                           0.91        88
   macro avg       0.88      0.88      0.88        88
weighted avg       0.91      0.91      0.91        88

Accuracy: 0.9090909090909091
Confusion matrix:

[[61  4]
 [ 4 19]]


Podemos ver que as métricas obtidas utilizando um modelo com todas as features foram as mesmas das obtidas com um modelo utilizando apenas Detergents_Paper, ou seja conseguimos resultados iguais com um modelo muito mais simples, assim economizando tempo e poder computacional.

## Ex 9

Nesse exercício o usuário deve ser capaz de entrar com valores para que o modelo preveja qual o canal de vendas a qual os dados pertencem.

In [66]:
prediction = NBmodel.predict([[int(input("Insira o valor de Detergents_Paper: "))]])
if prediction[0] == 0:
    print("Origem prevista é HoTeCa")
else:
    print("Origem prevista é Retail")

Insira o valor de Detergents_Paper:  3565


Origem prevista é Retail


C:\Users\lucas\anaconda3\envs\data_mining\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Vamos também fazer uma versão onde utilizamos a versão completa do modelo

In [67]:
input_features = []
input_features.append(int(input("Entre com o valor de Region :")))
input_features.append(int(input("Entre com o valor de Fresh :")))
input_features.append(int(input("Entre com o valor de Milk :")))
input_features.append(int(input("Entre com o valor de Grocery :")))
input_features.append(int(input("Entre com o valor de Frozen :")))
input_features.append(int(input("Entre com o valor de Detergents_Paper :")))
input_features.append(int(input("Entre com o valor de Delicatessen :")))

prediction = NBmodelFull.predict([input_features])
if prediction[0] == 0:
    print("Origem prevista é HoTeCa")
else:
    print("Origem prevista é Retail")

Entre com o valor de Region : 2
Entre com o valor de Fresh : 2
Entre com o valor de Milk : 2
Entre com o valor de Grocery : 2
Entre com o valor de Frozen : 2
Entre com o valor de Detergents_Paper : 2
Entre com o valor de Delicatessen : 2


Origem prevista é HoTeCa


C:\Users\lucas\anaconda3\envs\data_mining\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
